# Imports

In [135]:
import pandas as pd
import geopandas as gpd
import os
import requests

# Access geojson

In [136]:
#data from national grid https://www.nationalgrideso.com/data-portal/gis-boundaries-gb-dno-license-areas
filename = "national_grid_dno_regions_2024.geojson"
path_to_data = os.path.join( "..", "data")
filepath = os.path.join(path_to_data, filename)

# Read the GeoJSON file
uk_regions_2024 = gpd.read_file(filepath)

## rename some areas to match the region names provided by the API

This is so that sorting alphabetically will work nicely

In [98]:
rename_geojson_areas_to_api = {"Southern England":  "South England", "South and Central Scotland" : "South Scotland"}
uk_regions_2024["Area"] = uk_regions_2024["Area"].map(lambda x: rename_geojson_regions_to_match_api.get(x,x) )
uk_regions_2024

,ID,Name,DNO,Area,DNO_Full,geometry
0,10,_A,UKPN,East England,UK Power Networks,"MULTIPOLYGON (((599568.147 183257, 599695.147 ..."
1,11,_B,NGED,East Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((552880 326281, 552887.419 3262..."
2,12,_C,UKPN,London,UK Power Networks,"MULTIPOLYGON (((522000.864 178071.011, 521936...."
3,13,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,"MULTIPOLYGON (((250130 390270, 250090 390250, ..."
4,14,_E,NGED,West Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((402835.341 368841.642, 402984...."
5,15,_F,NPG,North East England,Northern Powergrid,"MULTIPOLYGON (((425411 637102, 425373 637102, ..."
6,16,_G,ENWL,North West England,Electricity North West,"MULTIPOLYGON (((357929 583757, 358217 582616, ..."
7,17,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,"MULTIPOLYGON (((201985.002 598540.625, 201777...."
8,19,_J,UKPN,South East England,UK Power Networks,"MULTIPOLYGON (((578235.16 179751.119, 578719.4..."
9,21,_K,NGED,South Wales,National Grid Electricity Distribution,"MULTIPOLYGON (((322590 164660, 322520 164620, ..."


In [103]:
lineup_file_df = uk_regions_2024.sort_values(by="Area").rename(columns={"ID" : "geojson_id", "Area" : "geojson_name"}).reset_index(drop=True)
lineup_api_df = region_response_df.iloc[:14, :3].sort_values(by="api_name").rename(columns={"id":"api_id"}).reset_index(drop=True)

lineup_df = pd.concat([lineup_file_df,lineup_api_df], axis=1)[["geojson_id","api_id", "geojson_name", "api_name"]]
lineup_df

,geojson_id,api_id,geojson_name,api_name
0,10,10,East England,East England
1,11,9,East Midlands,East Midlands
2,12,13,London,London
3,15,4,North East England,North East England
4,17,1,North Scotland,North Scotland
5,13,6,"North Wales, Merseyside and Cheshire",North Wales & Merseyside
6,16,3,North West England,North West England
7,19,14,South East England,South East England
8,20,12,South England,South England
9,18,2,South Scotland,South Scotland


In [118]:
id_map_dict = {}
for geo_id, api_id,api_name in zip(lineup_df.geojson_id, lineup_df.api_id,lineup_df.api_name):
    print(f"Region: {api_name}. Geojson id: {geo_id} -> Api_id: {api_id}" )
    id_map_dict[geo_id] = [api_id, api_name]
id_map_dict

Region: East England. Geojson id: 10 -> Api_id: 10
Region: East Midlands. Geojson id: 11 -> Api_id: 9
Region: London. Geojson id: 12 -> Api_id: 13
Region: North East England. Geojson id: 15 -> Api_id: 4
Region: North Scotland. Geojson id: 17 -> Api_id: 1
Region: North Wales & Merseyside. Geojson id: 13 -> Api_id: 6
Region: North West England. Geojson id: 16 -> Api_id: 3
Region: South East England. Geojson id: 19 -> Api_id: 14
Region: South England. Geojson id: 20 -> Api_id: 12
Region: South Scotland. Geojson id: 18 -> Api_id: 2
Region: South Wales. Geojson id: 21 -> Api_id: 7
Region: South West England. Geojson id: 22 -> Api_id: 11
Region: West Midlands. Geojson id: 14 -> Api_id: 8
Region: Yorkshire. Geojson id: 23 -> Api_id: 5


{10: [10, 'East England'],
 11: [9, 'East Midlands'],
 12: [13, 'London'],
 15: [4, 'North East England'],
 17: [1, 'North Scotland'],
 13: [6, 'North Wales & Merseyside'],
 16: [3, 'North West England'],
 19: [14, 'South East England'],
 20: [12, 'South England'],
 18: [2, 'South Scotland'],
 21: [7, 'South Wales'],
 22: [11, 'South West England'],
 14: [8, 'West Midlands'],
 23: [5, 'Yorkshire']}

In [108]:
uk_regions_2024

,ID,Name,DNO,Area,DNO_Full,geometry
0,10,_A,UKPN,East England,UK Power Networks,"MULTIPOLYGON (((599568.147 183257, 599695.147 ..."
1,11,_B,NGED,East Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((552880 326281, 552887.419 3262..."
2,12,_C,UKPN,London,UK Power Networks,"MULTIPOLYGON (((522000.864 178071.011, 521936...."
3,13,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,"MULTIPOLYGON (((250130 390270, 250090 390250, ..."
4,14,_E,NGED,West Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((402835.341 368841.642, 402984...."
5,15,_F,NPG,North East England,Northern Powergrid,"MULTIPOLYGON (((425411 637102, 425373 637102, ..."
6,16,_G,ENWL,North West England,Electricity North West,"MULTIPOLYGON (((357929 583757, 358217 582616, ..."
7,17,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,"MULTIPOLYGON (((201985.002 598540.625, 201777...."
8,19,_J,UKPN,South East England,UK Power Networks,"MULTIPOLYGON (((578235.16 179751.119, 578719.4..."
9,21,_K,NGED,South Wales,National Grid Electricity Distribution,"MULTIPOLYGON (((322590 164660, 322520 164620, ..."


In [ ]:
uk_regions_2024["ID"] = uk_regions_2024["ID"].map(lambda id : id_dict[id][0])

In [126]:
uk_regions_2024_processed = uk_regions_2024.sort_values(by="ID")

## Save to file

In [127]:
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html
# gdf.to_file('dataframe.geojson', driver='GeoJSON')  

In [130]:
filename_out = "uk_dno_regions_processed.geojson"
path_to_data = os.path.join( "..", "data")
filepath_out = os.path.join(path_to_data, filename)
print(filepath_out)

../data/national_grid_dno_regions_2024.geojson


In [131]:
uk_regions_2024_processed.to_file(filepath_out, driver="GeoJSON")

## Get live regional data to df

In [ ]:
url = "https://api.carbonintensity.org.uk/regional"
response = requests.get(url).json()
regions = response['data'][0]['regions']
regions;

In [116]:
def regional_response_to_df(response_regional):
    regions = response_regional['data'][0]['regions']

    rows = []
    for region in regions:
        region_id = region["regionid"]
        dno_region = region["dnoregion"]
        name = region["shortname"]
        intensity_forecast = region["intensity"]["forecast"]
        intensity_index = region["intensity"]["index"]
        
        row_dict = {
            "id" : region_id,
            "dno_region" : dno_region,
            "api_name" : name,
            "intensity_forecast" : intensity_forecast,
            "intensity_index" : intensity_index
        }
    
        for fuel in region["generationmix"]:
            fuel_name = fuel["fuel"]
            fuel_percentage = fuel["perc"]
            row_dict[fuel_name+"_perc"] = fuel_percentage
            
        rows.append(row_dict)
    
    return pd.DataFrame(rows)
region_response_df = regional_response_to_df(response)
region_response_df.columns

Index(['id', 'dno_region', 'api_name', 'intensity_forecast', 'intensity_index',
       'biomass_perc', 'coal_perc', 'imports_perc', 'gas_perc', 'nuclear_perc',
       'other_perc', 'hydro_perc', 'solar_perc', 'wind_perc'],
      dtype='object')

## Ye olde file way

In [ ]:
filename = "dno_regions.geojson"
path_to_data = os.path.join( "..", "data")
filepath = os.path.join(path_to_data, filename)

# Read the GeoJSON file
uk_regions = gpd.read_file(filepath)
uk_regions

In [133]:
! ls

API_formatting.ipynb  Geo_testing.ipynb
